In [25]:
import praw
from datetime import datetime, timedelta, timezone
import math

In [26]:
client_id = "e8O8uDpRkFbES6y2ZA9XwQ"
client_secret="5LSSWHnkHVM5-BIkxpgYlu8fKIqnzw"
user_agent = "Decaying Window"

In [27]:

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

In [28]:
def calculate_score(interactions, time_weight):
    """
    Calculate the score of a post based on its interactions and time weight.
    """
    return interactions * time_weight

In [29]:
def find_popular_posts(c, threshold, subreddit_name, time_window_hours):
    """
    Find popular posts in a subreddit within a specified time window.
    """
    subreddit = reddit.subreddit(subreddit_name)

    if time_window_hours <= 1:
        time_filter = 'hour'
    elif time_window_hours <= 24:
        time_filter = 'day'
    elif time_window_hours <= 168:
        time_filter = 'week'
    elif time_window_hours <= 720:
        time_filter = 'month'
    else:
        time_filter = 'year'

    current_time = datetime.now(timezone.utc)
    start_time = current_time - timedelta(hours=time_window_hours)

    popular_posts = []

    for post in subreddit.top(time_filter=time_filter, limit=None):
        post_time = datetime.fromtimestamp(post.created_utc, tz=timezone.utc)
        if post_time >= start_time:
            time_weight = math.ceil((1 - c) ** ((current_time - post_time).total_seconds() / 3600 + 1))
            score = calculate_score(post.score, time_weight)
            if score >= threshold:
                popular_posts.append((post.title, score))

    popular_posts.sort(key=lambda x: x[1], reverse=True)
    return popular_posts

In [30]:
c = 1e-6
threshold = 10
subreddit_name = 'IPL'
time_window_hours = 51

In [31]:
popular_posts = find_popular_posts(c, threshold, subreddit_name, time_window_hours)

In [32]:
print(f"Popular posts in r/{subreddit_name} within the last {time_window_hours} hour(s):")
for i, (title, score) in enumerate(popular_posts, start=1):
    print(f"{i}. {title} - Score: {score}")

Popular posts in r/IPL within the last 51 hour(s):
1. What a fiery spell from Mayank Yadav. That is some serious pace on debut today. Hope he doesn't fizzle out like Umran Malik - Score: 2316
2. Matheesha Pathirana has pulled off one of the catches of the season already vs Delhi! Wow! - Score: 1913
3. Ravichandran Ashwin says about hate against Hardik Pandya. - Score: 1053
4. Impossible for RCB to win the IPL with this bowling attack' - Michael Vaughan - Score: 923
5.  He Still Got It.  37(16)*🔥 - Score: 895
6. Move aside The OG king is back..  - Score: 759
7. Google has this little easter egg when you search "How to roast a KKR fan" - Score: 516
8. Accountability : Hardik Pandya - Score: 447
9. Banned from a certain sub for this - Score: 439
10. Why Chinnaswamy holds ipl matchs when bengluru is facing a water crisis. Since stadiums require a lot of water.  - Score: 365
11. Unpopular opinion - Score: 357
12. Thala is ❤️ - Score: 345
13. Gujarat Titans won by 7 wickets against Sunrisers